In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

2024-05-27 16:59:02.339739: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 16:59:02.339770: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 16:59:02.339790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 16:59:02.344622: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 16:59:02.967337: W tensorflow/compiler/

In [2]:
nltk.download('punkt')  
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/hao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hao/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hao/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# function define
def contributions_range(value):
    if value <= 5:
        return 1
    else:
        return 2
    
def hotel_star_range(value):
    if value <= 2.0:
        return 1
    elif value >= 2.5 and value <= 3.0:
        return 1
    elif value >= 3.5 and value <= 4.0:
        return 2
    else:
        return 2
    
# text preprocessing
def decontracting(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"don\'t", "do not", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'bout", "about", text)
    text = re.sub(r"\'til", "until", text)
    return text

stopwords_list = stopwords.words('english')
def remove_stopwords(tokens):
    texts = [i for i in tokens if i not in stopwords_list]
    return texts


def lemmatization(tokens):
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  return lemmatized_tokens


def word_preprocess(df, column_name):
  #lowercase
  df[column_name] = df[column_name].apply(lambda x: str(x).lower())

  #decontracting
  df[column_name] = df[column_name].apply(decontracting)

  #remove tags, punctuations, numbers
  df[column_name] = df[column_name].apply(lambda x: re.sub('[^a-zA-Z!]', ' ', x))

  #tokenization
  import nltk
  nltk.download('punkt')
  df[column_name] = df[column_name].apply(lambda x: nltk.word_tokenize(x))

  #remove stopwords
  df[column_name] = df[column_name].apply(remove_stopwords)
  return df

In [4]:
df = pd.read_csv('../../Final_Datasets/TA_combined_df_City_tourism_type_VADER_final_Stanza.csv')
df

,Hotel_locID,Hotel_geoID_x,Review_id,Review_Date,Date_of_stay,Reviewer_Contributions,Reviewer_helpful_vote,Review_Rating,Trip_type,Review_helpful_votes,...,Resaurant_count,Attractions_count,Hotel_styles,Popular_mentions,WordCount,Title,Review,Compound_Score,Unreliable,Stanza_Score
0,23448880,60763,916928494,Sep 16,September 2023,0,0,1,Traveled on business,1,...,348.0,100.0,['Business'],"['front desk', 'landmark view', 'city view roo...",66,Unfriendly staff and dirty,I travel a lot - and I am in general very flex...,-0.4095,0,0
1,23448880,60763,828890910,Feb 2022,February 2022,45,26,5,NaN,8,...,348.0,100.0,['Business'],"['front desk', 'landmark view', 'city view roo...",115,Perfect for Us,We recently chose Motto for an overnight in NY...,0.9903,0,2
2,23448880,60763,915189618,Sep 6,September 2023,0,0,3,Traveled on business,0,...,348.0,100.0,['Business'],"['front desk', 'landmark view', 'city view roo...",99,Not satisfied with the overall experience,The rooms are two small for the price that you...,-0.0121,0,0
3,23448880,60763,915010751,Sep 5,September 2023,68,88,5,Traveled with family,1,...,348.0,100.0,['Business'],"['front desk', 'landmark view', 'city view roo...",152,Magnificent Motto - fantastic staff,I have just returned home after a five day sta...,0.9746,0,2
4,23448880,60763,914794870,Sep 4,August 2023,179,68,5,Traveled solo,0,...,348.0,100.0,['Business'],"['front desk', 'landmark view', 'city view roo...",201,Worthy of a 5-star rating!,I did not believe the overwhelming 5-star revi...,0.9949,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65977,12841013,35805,536016764,Oct 2017,October 2017,0,0,4,Traveled solo,0,...,209.0,90.0,[],[],107,Great cost benefit,I stayed in the loop suites from 10/19/17 to 1...,0.9891,0,2
65978,12841013,35805,535664344,Oct 2017,October 2017,0,0,5,Traveled solo,0,...,209.0,90.0,[],[],43,AWESOME,I enjoy my stay! Very nice and very clean and ...,0.9703,0,2
65979,12841013,35805,535094403,Oct 2017,January 2017,0,0,5,Traveled solo,0,...,209.0,90.0,[],[],46,"Very good hotel, highly recommended",I've stayed in this apartment hotel in January...,0.7639,0,2
65980,12841013,35805,531949578,Oct 2017,May 2017,0,0,1,Traveled as a couple,0,...,209.0,90.0,[],[],158,Shocking,"We booked an apartment for 4 people, 2 couples...",-0.5707,0,0


In [5]:
df_processed = df.dropna(subset=['Hotel_star'])

In [6]:
# create new columns
df_processed['Reviewer_Contributions_range'] = df_processed['Reviewer_Contributions'].apply(contributions_range)
df_processed['Hotel_star_range'] = df_processed['Hotel_star'].apply(hotel_star_range)
df_processed

/tmp/ipykernel_462673/1132114254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['Reviewer_Contributions_range'] = df_processed['Reviewer_Contributions'].apply(contributions_range)
/tmp/ipykernel_462673/1132114254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed['Hotel_star_range'] = df_processed['Hotel_star'].apply(hotel_star_range)


,Hotel_locID,Hotel_geoID_x,Review_id,Review_Date,Date_of_stay,Reviewer_Contributions,Reviewer_helpful_vote,Review_Rating,Trip_type,Review_helpful_votes,...,Hotel_styles,Popular_mentions,WordCount,Title,Review,Compound_Score,Unreliable,Stanza_Score,Reviewer_Contributions_range,Hotel_star_range
0,23448880,60763,916928494,Sep 16,September 2023,0,0,1,Traveled on business,1,...,['Business'],"['front desk', 'landmark view', 'city view roo...",66,Unfriendly staff and dirty,I travel a lot - and I am in general very flex...,-0.4095,0,0,1,2
1,23448880,60763,828890910,Feb 2022,February 2022,45,26,5,NaN,8,...,['Business'],"['front desk', 'landmark view', 'city view roo...",115,Perfect for Us,We recently chose Motto for an overnight in NY...,0.9903,0,2,2,2
2,23448880,60763,915189618,Sep 6,September 2023,0,0,3,Traveled on business,0,...,['Business'],"['front desk', 'landmark view', 'city view roo...",99,Not satisfied with the overall experience,The rooms are two small for the price that you...,-0.0121,0,0,1,2
3,23448880,60763,915010751,Sep 5,September 2023,68,88,5,Traveled with family,1,...,['Business'],"['front desk', 'landmark view', 'city view roo...",152,Magnificent Motto - fantastic staff,I have just returned home after a five day sta...,0.9746,0,2,2,2
4,23448880,60763,914794870,Sep 4,August 2023,179,68,5,Traveled solo,0,...,['Business'],"['front desk', 'landmark view', 'city view roo...",201,Worthy of a 5-star rating!,I did not believe the overwhelming 5-star revi...,0.9949,0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65954,290675,35805,17172887,Jun 2008,June 2008,0,2,4,Traveled with family,0,...,[],[],78,Loved the Amber Inn,I loved staying at the Amber Inn!!! By me bein...,0.9112,0,2,1,1
65955,290675,35805,8498082,Aug 2007,August 2007,8,9,5,NaN,0,...,[],[],48,Cannot get any better for the price,This hotel I would recommend to anyone. The st...,0.9485,0,2,2,1
65956,290675,35805,6916950,Mar 2007,March 2007,0,7,1,Traveled with family,0,...,[],[],512,Disgusted,"On 03-03-07, my family and I got a room from T...",-0.9605,0,0,1,1
65957,290675,35805,6898967,Feb 2007,March 2007,0,0,5,Traveled on business,0,...,[],[],64,Wow! What a great place for the price!,I enjoyed my stay at the Amber Inn near souths...,0.9601,0,2,1,1


In [7]:
# Split 4 sub-datasets
LCLS = df_processed[(df_processed['Reviewer_Contributions_range'] == 1) & (df_processed['Hotel_star_range'] == 1)]
LCHS = df_processed[(df_processed['Reviewer_Contributions_range'] == 1) & (df_processed['Hotel_star_range'] == 2)]
HCLS = df_processed[(df_processed['Reviewer_Contributions_range'] == 2) & (df_processed['Hotel_star_range'] == 1)]
HCHS = df_processed[(df_processed['Reviewer_Contributions_range'] == 2) & (df_processed['Hotel_star_range'] == 2)]

# split unreliable reviews
LCLS_unreliable = LCLS[LCLS['Unreliable'] == 1]
LCHS_unreliable = LCHS[LCHS['Unreliable'] == 1]
HCLS_unreliable = HCLS[HCLS['Unreliable'] == 1]
HCHS_unreliable = HCHS[HCHS['Unreliable'] == 1]

In [8]:
# save to csv
# LCLS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/LCLS_unreliable.csv', index=False)
# LCHS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/LCHS_unreliable.csv', index=False)
# HCLS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/HCLS_unreliable.csv', index=False)
# HCHS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/HCHS_unreliable.csv', index=False)

In [9]:
# text preprocessing
# LCLS_unreliable_text_preprocessed_df = word_preprocess(LCLS_unreliable,'Review')
# LCHS_unreliable_text_preprocessed_df = word_preprocess(LCHS_unreliable,'Review')
# HCLS_unreliable_text_preprocessed_df = word_preprocess(HCLS_unreliable,'Review')
# HCHS_unreliable_text_preprocessed_df = word_preprocess(HCHS_unreliable,'Review')

In [10]:
LCLS_unreliable['Review']

518      The room is too smaller for the high price, an...
548      Clogged sink would not drain. I appreciated th...
1411     Was an okay place to stay but we found a coupl...
1503     Stayed here for 4 nights from 1 Sept. On arriv...
2028     Overall this was an average hotel stay. We had...
                               ...                        
62939    The employee at the front desk was extremely p...
63132    Flight was cancelled leaving me scrambling to ...
65607    I was expecting a room resembling the image yo...
65685    This hotel is well located but our room was ol...
65863    The rooms were large, clean and the staff was ...
Name: Review, Length: 182, dtype: object

In [11]:
# split unreliable dataset into 2 columns
selected_columns_unreliable = ['Review', 'Review_Rating']
LCLS_text_unreliable = LCLS_unreliable.loc[:, selected_columns_unreliable]
LCHS_text_unreliable = LCHS_unreliable.loc[:, selected_columns_unreliable]
HCLS_text_unreliable = HCLS_unreliable.loc[:, selected_columns_unreliable]
HCHS_text_unreliable = HCHS_unreliable.loc[:, selected_columns_unreliable]

# reset index
LCLS_text_unreliable.reset_index(drop=True, inplace=True)
LCHS_text_unreliable.reset_index(drop=True, inplace=True)
HCLS_text_unreliable.reset_index(drop=True, inplace=True)
HCHS_text_unreliable.reset_index(drop=True, inplace=True)

# let reviews in each row become a string
# LCLS_text_unreliable['Review'] = [' '.join(text) for text in LCLS_text_unreliable['Review']]
# LCHS_text_unreliable['Review'] = [' '.join(text) for text in LCHS_text_unreliable['Review']]
# HCLS_text_unreliable['Review'] = [' '.join(text) for text in HCLS_text_unreliable['Review']]
# HCHS_text_unreliable['Review'] = [' '.join(text) for text in HCHS_text_unreliable['Review']]

In [12]:
LCLS_unreliable_X = LCLS_text_unreliable['Review']
LCLS_unreliable_y = LCLS_text_unreliable['Review_Rating']

LCHS_unreliable_X = LCHS_text_unreliable['Review']
LCHS_unreliable_y = LCHS_text_unreliable['Review_Rating']

HCLS_unreliable_X = HCLS_text_unreliable['Review']
HCLS_unreliable_y = HCLS_text_unreliable['Review_Rating']

HCHS_unreliable_X = HCHS_text_unreliable['Review']
HCHS_unreliable_y = HCHS_text_unreliable['Review_Rating']

In [13]:
LCLS_unreliable_X

0      The room is too smaller for the high price, an...
1      Clogged sink would not drain. I appreciated th...
2      Was an okay place to stay but we found a coupl...
3      Stayed here for 4 nights from 1 Sept. On arriv...
4      Overall this was an average hotel stay. We had...
                             ...                        
177    The employee at the front desk was extremely p...
178    Flight was cancelled leaving me scrambling to ...
179    I was expecting a room resembling the image yo...
180    This hotel is well located but our room was ol...
181    The rooms were large, clean and the staff was ...
Name: Review, Length: 182, dtype: object

## Longformer

In [14]:
import transformers
import torch

/home/hao/Code/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# BERT(longformer) model
model_name = 'allenai/longformer-base-4096'
tokenizer = transformers.LongformerTokenizer.from_pretrained(model_name)
model = transformers.LongformerModel.from_pretrained(model_name).to(device)

In [17]:
# define function
def bert_embedding(X_train):
    embeddings = []
    for text in X_train:
        # 將文本轉成BERT的輸入格式，即加上[CLS]與[SEP] token，並轉成tensor
        encoded_text = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt').to(device)

        # 用預訓練BERT模型轉成向量
        with torch.no_grad():
            model_output = model(encoded_text['input_ids'], attention_mask=encoded_text['attention_mask'])

        # 取出[CLS] token對應的向量作為整個文本的向量表示
        embeddings.append(model_output.last_hidden_state[:, 0, :].squeeze().tolist())
    return embeddings

In [18]:
LCLS_unreliable_X_bert = pd.DataFrame(bert_embedding(LCLS_unreliable_X))
LCHS_unreliable_X_bert = pd.DataFrame(bert_embedding(LCHS_unreliable_X))
HCLS_unreliable_X_bert = pd.DataFrame(bert_embedding(HCLS_unreliable_X))
HCHS_unreliable_X_bert = pd.DataFrame(bert_embedding(HCHS_unreliable_X))

In [19]:
LCLS_unreliable_X_bert

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.050908,0.122207,0.006951,-0.149705,0.163385,-0.041357,-0.021272,0.020481,0.025886,-0.144073,...,0.014479,-0.007821,-0.062212,-0.041804,-0.013096,0.087481,0.079315,-0.131910,-0.050172,-0.028143
1,-0.056304,0.065346,-0.003244,-0.103457,0.136883,-0.013560,-0.020571,0.073960,0.031345,-0.118462,...,0.086262,-0.035446,-0.161022,-0.100020,0.057332,0.129222,0.014672,-0.127074,-0.032873,-0.039966
2,-0.059939,0.077162,0.005043,-0.143026,0.109789,-0.001913,-0.045074,-0.008334,0.051527,-0.163747,...,0.044862,-0.050243,-0.136801,-0.030328,-0.012736,0.077838,0.090204,-0.165169,0.004914,-0.010073
3,-0.044114,0.083771,0.060063,-0.152079,0.033083,-0.036279,-0.024605,0.004348,0.105753,-0.191153,...,0.023283,-0.076884,-0.104329,0.005776,0.050605,0.058708,0.029266,-0.070925,0.014324,-0.026834
4,-0.107308,0.103177,0.049286,-0.159762,0.069486,-0.017025,-0.025463,0.031742,0.081381,-0.152819,...,0.002522,-0.029864,-0.105313,-0.019829,0.019185,0.094003,0.010118,-0.122194,-0.008311,-0.015755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,-0.109419,0.106436,0.056269,-0.120804,0.112206,0.020836,-0.040967,0.016013,0.089292,-0.177498,...,0.057258,-0.031642,-0.133268,-0.048638,0.009734,0.082164,0.002466,-0.141618,-0.045946,0.026640
178,-0.086621,0.051575,0.021621,-0.111164,0.131050,0.005625,-0.017727,0.037761,0.080737,-0.159835,...,0.013383,-0.010462,-0.101075,-0.048530,-0.012145,0.119106,-0.023878,-0.144482,-0.068421,-0.021840
179,-0.044492,0.065558,-0.007216,-0.124660,0.140513,-0.048782,-0.033654,0.032306,0.046382,-0.144419,...,-0.000786,-0.024157,-0.090325,-0.051407,-0.050224,0.090551,0.034886,-0.127112,-0.019305,-0.048137
180,-0.079327,0.118288,0.031631,-0.180782,0.114745,-0.039249,-0.037943,0.012342,0.053028,-0.152206,...,0.049621,-0.046756,-0.082948,-0.019594,-0.007275,0.074594,0.035064,-0.129477,-0.003062,-0.007179


In [20]:
input_data = LCLS_unreliable_X_bert
# 步驟 1：加載模型
model = load_model('../../Experiment_code/Model/LCLS_lstm_model_longformer.h5')

# 步驟 2：加載輸入數據
input_data = input_data.to_numpy().reshape(input_data.shape[0], 1, input_data.shape[1])

# 步驟 3：使用模型進行預測
predictions = model.predict(input_data)

predictions_flat = predictions.flatten()
LCLS_unreliable['Predicted_Rating'] = predictions_flat
LCLS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/LCLS_unreliable.csv', index=False)

# 輸出預測結果
# print(predictions)

2024-05-27 16:59:34.484334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-27 16:59:34.501992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-27 16:59:34.502216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[[[1.6141143 ]]

 [[0.6785921 ]]

 [[3.2872508 ]]

 [[2.4850705 ]]

 [[3.2471387 ]]

 [[1.9488622 ]]

 [[1.7927523 ]]

 [[3.803065  ]]

 [[3.5644584 ]]

 [[4.7487864 ]]

 [[1.3126128 ]]

 [[1.5054604 ]]

 [[1.7215164 ]]

 [[4.6616287 ]]

 [[4.4065595 ]]

 [[2.8291013 ]]

 [[3.8771217 ]]

 [[1.0389017 ]]

 [[5.097446  ]]

 [[1.4915597 ]]

 [[3.695934  ]]

 [[2.0425231 ]]

 [[0.89543724]]

 [[2.5256207 ]]

 [[4.240047  ]]

 [[2.7910206 ]]

 [[1.6878786 ]]

 [[0.5661218 ]]

 [[0.4646063 ]]

 [[4.350291  ]]

 [[2.8699994 ]]

 [[3.4918256 ]]

 [[3.8369381 ]]

 [[4.652214  ]]

 [[2.2640462 ]]

 [[3.9201448 ]]

 [[4.1602798 ]]

 [[4.0899577 ]]

 [[2.086515  ]]

 [[3.3712265 ]]

 [[1.5742192 ]]

 [[1.4520483 ]]

 [[2.5481663 ]]

 [[3.9126039 ]]

 [[1.6595402 ]]

 [[0.92838454]]

 [[1.9017489 ]]

 [[0.93843794]]

 [[4.5024934 ]]

 [[1.2769547 ]]

 [[2.1618092 ]]

 [[1.2211084 ]]

 [[2.0998194 ]]

 [[4.539648  ]]

 [[2.4164038 ]]

 [[1.9469227 ]]

 [[2.74248

/tmp/ipykernel_462673/3408578930.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LCLS_unreliable['Predicted_Rating'] = predictions_flat


In [21]:
input_data = LCHS_unreliable_X_bert
# 步驟 1：加載模型
model = load_model('../../Experiment_code/Model/LCHS_lstm_model_longformer.h5')

# 步驟 2：加載輸入數據
input_data = input_data.to_numpy().reshape(input_data.shape[0], 1, input_data.shape[1])

# 步驟 3：使用模型進行預測
predictions = model.predict(input_data)

predictions_flat = predictions.flatten()
LCHS_unreliable['Predicted_Rating'] = predictions_flat
LCHS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/LCHS_unreliable.csv', index=False)

# 輸出預測結果
# print(predictions)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/tmp/ipykernel_462673/2234562453.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LCHS_unreliable['Predicted_Rating'] = predictions_flat


In [23]:
input_data = HCLS_unreliable_X_bert
# 步驟 1：加載模型
model = load_model('../../Experiment_code/Model/HCLS_lstm_model_longformer.h5')

# 步驟 2：加載輸入數據
input_data = input_data.to_numpy().reshape(input_data.shape[0], 1, input_data.shape[1])

# 步驟 3：使用模型進行預測
predictions = model.predict(input_data)

predictions_flat = predictions.flatten()
HCLS_unreliable['Predicted_Rating'] = predictions_flat
HCLS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/HCLS_unreliable.csv', index=False)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/tmp/ipykernel_462673/1712698182.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCLS_unreliable['Predicted_Rating'] = predictions_flat


In [24]:
input_data = HCHS_unreliable_X_bert
# 步驟 1：加載模型
model = load_model('../../Experiment_code/Model/HCHS_lstm_model_longformer.h5')

# 步驟 2：加載輸入數據
input_data = input_data.to_numpy().reshape(input_data.shape[0], 1, input_data.shape[1])

# 步驟 3：使用模型進行預測
predictions = model.predict(input_data)

predictions_flat = predictions.flatten()
HCHS_unreliable['Predicted_Rating'] = predictions_flat
HCHS_unreliable.to_csv('../../Final_Datasets/Unreliable_datasets/HCHS_unreliable.csv', index=False)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


/tmp/ipykernel_462673/1384022494.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCHS_unreliable['Predicted_Rating'] = predictions_flat
